### Формально, это не совсем ДЗ №5
Это слабо-успешные попытки завести lightfm и получить хоть что-нибудь годное из него на наших данных.
- Просто - плюсы - минусы гибридных систем мы уже обсуждали на вебинарах, 
- Про hh мы тоже уже читали и обсуждали

По итогу - сколько нибудь близкого к коллаборативным методам качества получить не удалось
И остается загадкой - что делать с нехваткой данных по пользователям :)

In [427]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

In [174]:
data = pd.read_csv('data/retail_train.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [179]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [185]:
user_features.shape

(801, 8)

In [ ]:
# Houston, у нас проблема - у нас 2499 пользователей, а описания есть на 801

In [186]:
item_features.shape

(92353, 7)

In [190]:
data.item_id.unique().shape

(89051,)

In [191]:
# С айтем фичами наверное все ок их даже больше - хотя никто не гарантирует что они от тех товаров которые у нас есть

In [114]:
# Тестовые таргеты для получения метрик
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [532]:
# Зарезаем до top_5000
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

/Users/postas/Anaconda/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [6]:
item_features['department'].unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [193]:
%%time
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

CPU times: user 2.11 s, sys: 299 ms, total: 2.41 s
Wall time: 2.46 s


In [194]:
user_item_matrix = user_item_matrix.astype(float)

In [278]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [198]:
# Юзер фичи. На вебинаре было подозрение что фичи пользователей не попадут в своих пользователей
# Но судя по результату мы видим, что все попало как надо. Просто фичи у нас не на всех пользователей есть
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
# Итем фичи
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
202291,69.0,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
397896,69.0,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,


In [ ]:
# Здесь тоже все ок. 

In [212]:
# OHE
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [320]:
model = LightFM(no_components=5,
                loss='bpr', # 'warp'
                learning_rate=0.05, 
                item_alpha=0.1, user_alpha=0.1, 
                random_state=42)

In [321]:
%%time
model.fit((csr_matrix(user_item_matrix).tocsr() > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, num_threads=4)

CPU times: user 12.4 s, sys: 165 ms, total: 12.5 s
Wall time: 13.1 s


In [322]:
precision_at_k(model, csr_matrix(user_item_matrix).tocsr(), 
                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                 k=5).mean()

0.3611845

In [323]:
user_emb = model.get_user_representations(
    features=csr_matrix(user_feat_lightfm.values).tocsr())

In [324]:
user_emb

(array([-13.852102,   0.      ,   0.      , ..., -14.19927 ,  -9.830172,
          0.      ], dtype=float32),
 array([[-5.2542461e-08, -8.6278980e-08,  1.9423593e-08,  2.6151032e-08,
          5.5134862e-08],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        ...,
        [ 1.3972255e-08, -5.7702863e-08,  2.2189147e-08, -1.2580411e-08,
         -6.4823640e-09],
        [ 2.8321541e-09, -3.0721097e-08,  3.5153239e-08, -2.0122251e-08,
          2.1327965e-08],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00]], dtype=float32))

In [325]:
# Так.. Надо поидее перемножить эмбеддинги пользователей и айтемов и прибавить туда суммы их байасов

$$ v = u \cdot i^T + b_u + b_i $$

Такая кажется была формула

In [326]:
item_emb = model.get_item_representations(
    features=csr_matrix(item_feat_lightfm.values).tocsr())

In [327]:
item_emb

(array([-1.1022573 , -0.79444575, -1.1022573 , ..., -1.7802383 ,
        -1.7643487 , -1.5184562 ], dtype=float32),
 array([[ 7.2097485e-08,  1.0526804e-07, -1.6642578e-07, -9.5143548e-08,
         -4.0880135e-08],
        [-2.1684573e-07,  7.7045542e-08,  6.5395511e-09, -4.3429336e-07,
          6.7984558e-08],
        [ 7.2097485e-08,  1.0526804e-07, -1.6642578e-07, -9.5143548e-08,
         -4.0880135e-08],
        ...,
        [ 1.4072265e-06,  2.3155333e-07, -1.4232554e-06,  1.0916584e-06,
          6.2832775e-08],
        [-9.2212395e-07, -7.0876865e-07, -3.3831842e-07, -4.9382083e-07,
          2.3914231e-06],
        [ 3.6599423e-07,  2.6292958e-07, -6.3063470e-07,  1.3884188e-07,
          1.6508851e-07]], dtype=float32))

### Получение рекомендаций путем самостоятельных вычислений на основе эмбеддингов

In [328]:
# Сигмоида. Возможно и не понадобиться - и так можно будет отсортировать
def sig(x):
    return 1/(1 + np.exp(-x))
v_sig = np.vectorize(sig)

In [329]:
# Множим u на i
embs_matrix = user_emb[1] @ item_emb[1].T

In [349]:
%%time
# Добавляем байасы b_u и b_i
for u in range(embs_matrix.shape[0]):
    for i in range(embs_matrix.shape[1]):
        embs_matrix[u][i] += user_emb[0][u] + item_emb[0][i]

CPU times: user 11 s, sys: 44.7 ms, total: 11 s
Wall time: 11.2 s


In [350]:
# Сигмоида (вероятности)
# Опустим этот шаг пока
fast_recs = embs_matrix
# fast_recs = v_sig(embs_matrix)

In [351]:
def to_item_id(idx):
    return id_to_itemid[idx]

v_to_item_id = np.vectorize(to_item_id)

In [352]:
%%time
result['fast_lightfm'] = result['user_id'].apply(lambda x: 
                                             v_to_item_id(np.argsort(fast_recs[userid_to_id[x]])[-5:]))

CPU times: user 619 ms, sys: 9.62 ms, total: 628 ms
Wall time: 713 ms


In [360]:
result.head()

,user_id,actual,fast_lightfm
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5703832, 397896, 1404121, 1082185, 999999]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5703832, 397896, 1404121, 1082185, 999999]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5703832, 397896, 1404121, 1082185, 999999]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5703832, 397896, 1404121, 1082185, 999999]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5703832, 397896, 1404121, 1082185, 999999]"


In [337]:
# Что-то они все одинаковые.. ок. Какой получется pr_at_k
def precision_at_k(recommended, actual, k=5):
    flags = np.isin(recommended[:k], actual)
    return flags.sum() / len(recommended[:k])

v_precision_at_k = np.vectorize(precision_at_k)

In [338]:
v_precision_at_k(result.fast_lightfm, result.actual).mean()

0.07061704211557296

In [ ]:
# Ожидаемо. Возможно я что-то напутал в математике - есть штатный метод predict

### Штатный predict

In [365]:
recs = model.predict(1, matrix_itemids, 
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(), 
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              num_threads=4
             )
recs

array([-1.1022573 , -0.79444575, -1.1022573 , ..., -1.7802383 ,
       -1.7643487 , -1.5184562 ], dtype=float32)

In [367]:
v_to_item_id(np.argsort(recs))[-5:]

array([5703832,  397896, 1404121, 1082185,  999999])

In [368]:
# Опять те-же айтемы... настораживает. Ну ладно - сделаем для всех юзеров

In [369]:
# Для всех юзеров для всех айтемов (отбираем 5 с максимальным значением)

In [370]:
%%time
result['lightfm'] = result['user_id'].apply(lambda x: 
                        v_to_item_id(
                            np.argsort(
                                model.predict(int(userid_to_id[x]), matrix_itemids, 
                                              user_features=csr_matrix(user_feat_lightfm.values).tocsr(), 
                                              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                              num_threads=4)
                            )[-5:]))

CPU times: user 2min 58s, sys: 1.73 s, total: 3min
Wall time: 3min 7s


In [371]:
result.head(20)

,user_id,actual,fast_lightfm,lightfm
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]"
9,16,"[1062973, 1082185, 13007710]","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]"


In [372]:
v_precision_at_k(result.lightfm, result.actual).mean()

0.07061704211557296

In [373]:
# Все ровно точно также.. И результаты predict дал такие-же.. Что-то явно пошло не так раньше... до фита видимо

In [376]:
# Вобщем можно конечно предположить, что виной всему отсутствие фичей на две трети юзеров, если это причина - 
# то с нашими данными ловить нечего - ни в lightfm, ни в гибридных вариантах - поскольку не на чем будет 
# учить классифаеры (или ранкеры)

In [378]:
# Чисто теоретически можно попробовать пока любой произвольный датасет с достаточным количеством людей. 
# Понятно что это будет большая натяжка, но хотя-бы понять, в этом ли причина кривизны результатов lightfm

## Пробуем с исправным (но не относящимся к нашим интеракциям) набором юзеров

In [403]:
# Итак... Датасет про банковский скоринг )
user_features = pd.read_csv('data/course_project_train.csv')
user_features.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


In [404]:
user_features.shape

(7500, 17)

In [405]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Home Ownership                7500 non-null   object 
 1   Annual Income                 5943 non-null   float64
 2   Years in current job          7129 non-null   object 
 3   Tax Liens                     7500 non-null   float64
 4   Number of Open Accounts       7500 non-null   float64
 5   Years of Credit History       7500 non-null   float64
 6   Maximum Open Credit           7500 non-null   float64
 7   Number of Credit Problems     7500 non-null   float64
 8   Months since last delinquent  3419 non-null   float64
 9   Bankruptcies                  7486 non-null   float64
 10  Purpose                       7500 non-null   object 
 11  Term                          7500 non-null   object 
 12  Current Loan Amount           7500 non-null   float64
 13  Cur

In [406]:
user_features = user_features.dropna()
user_features.shape

(2585, 17)

In [407]:
# Если просто дропнуть все наны, он идеально подходит нам по размеру. 
# Но конечно никак не отражает взаимосвязей покупателей из нашей основной таблицы интерракций. 
# Кстати название нашего исходного датасета (hh_demographics) - тоже навевает мысль о том, что предложенные данные
# Не особо про нашу табличку покупок.. 
# (если думать что hh - это хедхантер - то возникает сильное подозрение что этот датасет взят совершенно случайно)

In [408]:
# user_id делаем из индекса конечно
user_features.reset_index(inplace=True, drop=True)
user_features['user_id'] = user_features.index
user_features.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default,user_id
0,Home Mortgage,1511108.0,10+ years,0.0,9.0,20.3,388124.0,0.0,73.0,0.0,home improvements,Short Term,99999999.0,51623.0,2317.0,745.0,0,0
1,Rent,1040060.0,10+ years,0.0,13.0,12.0,330374.0,0.0,18.0,0.0,other,Short Term,250888.0,89015.0,19761.0,705.0,1,1
2,Home Mortgage,1401744.0,2 years,0.0,9.0,29.0,387222.0,0.0,40.0,0.0,debt consolidation,Long Term,553586.0,201989.0,13434.0,695.0,1,2
3,Home Mortgage,1651993.0,10+ years,0.0,11.0,26.5,663894.0,0.0,44.0,0.0,debt consolidation,Long Term,585090.0,252852.0,8742.0,697.0,0,3
4,Home Mortgage,1047394.0,6 years,0.0,7.0,34.4,401104.0,0.0,45.0,0.0,debt consolidation,Short Term,324852.0,183597.0,7786.0,719.0,1,4


In [409]:
from sklearn.preprocessing import StandardScaler

In [413]:
# Не припомню - важна ли стандартизация для lightfm
scaler = StandardScaler()
features_to_stand = user_features.select_dtypes(include='float64').columns.tolist()
stand_features = scaler.fit_transform(user_features[features_to_stand])
user_features[features_to_stand] = pd.DataFrame(stand_features, columns=features_to_stand)

In [418]:
user_features.head(2)

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default,user_id
0,Home Mortgage,0.061469,10+ years,-0.122708,-0.527131,0.172735,-0.050125,-0.335664,1.776966,-0.324577,home improvements,Short Term,2.518650,-0.714691,-1.437282,-0.248652,0,0
1,Rent,-0.471701,10+ years,-0.122708,0.312317,-1.058208,-0.057758,-0.335664,-0.769450,-0.324577,other,Short Term,-0.398873,-0.586188,0.075077,-0.273726,1,1


In [419]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
user_id,,,,,,,,,,,,,,,,,
1,Rent,-0.471701,10+ years,-0.122708,0.312317,-1.058208,-0.057758,-0.335664,-0.769450,-0.324577,other,Short Term,-0.398873,-0.586188,0.075077,-0.273726,1
2,Home Mortgage,-0.062318,2 years,-0.122708,-0.527131,1.463001,-0.050245,-0.335664,0.249116,-0.324577,debt consolidation,Long Term,-0.390019,-0.197939,-0.473461,-0.279995,1


In [422]:
cat_features = user_features.select_dtypes(include='object').columns.tolist()
user_feat_lightfm = pd.get_dummies(user_feat, columns=cat_features)
user_feat_lightfm.head()

,Annual Income,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Current Loan Amount,Current Credit Balance,...,Purpose_major purchase,Purpose_medical bills,Purpose_moving,Purpose_other,Purpose_small business,Purpose_take a trip,Purpose_vacation,Purpose_wedding,Term_Long Term,Term_Short Term
user_id,,,,,,,,,,,,,,,,,,,,,
1,-0.471701,-0.122708,0.312317,-1.058208,-0.057758,-0.335664,-0.769450,-0.324577,-0.398873,-0.586188,...,0,0,0,1,0,0,0,0,0,1
2,-0.062318,-0.122708,-0.527131,1.463001,-0.050245,-0.335664,0.249116,-0.324577,-0.390019,-0.197939,...,0,0,0,0,0,0,0,0,1,0
3,0.220934,-0.122708,-0.107407,1.092235,-0.013680,-0.335664,0.434310,-0.324577,-0.389098,-0.023142,...,0,0,0,0,0,0,0,0,1,0
4,-0.463400,-0.122708,-0.946855,2.263856,-0.048410,-0.335664,0.480609,-0.324577,-0.396709,-0.261145,...,0,0,0,0,0,0,0,0,0,1
5,0.079275,-0.122708,1.571489,0.869776,0.070795,-0.335664,-0.954644,-0.324577,-0.398454,0.809642,...,0,0,0,0,0,0,0,0,0,1


In [423]:
# Как-то так.. Готовы попробовать завести lightfm снова

In [424]:
model = LightFM(no_components=5,
                loss='bpr', # 'warp'
                learning_rate=0.05, 
                item_alpha=0.1, user_alpha=0.1, 
                random_state=42)

In [425]:
%%time
model.fit((csr_matrix(user_item_matrix).tocsr() > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, num_threads=4)

CPU times: user 16.3 s, sys: 108 ms, total: 16.5 s
Wall time: 16.5 s


In [428]:
# Оверрайднули случайн этот метод
from lightfm.evaluation import precision_at_k, recall_at_k

precision_at_k(model, csr_matrix(user_item_matrix).tocsr(), 
                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                 k=5).mean()

0.3611845

In [ ]:
# Вот абсолютно ничего не поменялось... 
# Нет никакой зависимости от юзер-фичей здесь - хоть вообще нули туда подай - ничего не изменится
# вообще непонятно - как он тут дает какой-то скор - если для всех предсказывает одно и то-же

In [430]:
%%time
result['lightfm_user_feats'] = result['user_id'].apply(lambda x: 
                        v_to_item_id(
                            np.argsort(
                                model.predict(int(userid_to_id[x]), matrix_itemids, 
                                              user_features=csr_matrix(user_feat_lightfm.values).tocsr(), 
                                              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                              num_threads=4)
                            )[-5:]))

CPU times: user 2min 57s, sys: 386 ms, total: 2min 58s
Wall time: 2min 58s


In [431]:
result.head()

,user_id,actual,fast_lightfm,lightfm,lightfm_user_feats
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]"


In [432]:
# Один новый айтем появился.. остальное все также осталось. Для всех юзеров - все одно и то-же
# Уверен, что если мы вручную перемножим матрицы и добавим байасы - результат будет ровно таким-же
# Но - попробуем

### Вычисляем вручную

In [433]:
user_emb = model.get_user_representations(
    features=csr_matrix(user_feat_lightfm.values).tocsr())

In [442]:
item_emb = model.get_item_representations(
    features=csr_matrix(item_feat_lightfm.values).tocsr())

In [443]:
# Перемножаем ембеддинги
embs_matrix = user_emb[1] @ item_emb[1].T

In [444]:
%%time
# Добавляем байасы b_u и b_i
for u in range(embs_matrix.shape[0]):
    for i in range(embs_matrix.shape[1]):
        embs_matrix[u][i] += user_emb[0][u] + item_emb[0][i]

CPU times: user 10.9 s, sys: 24.9 ms, total: 10.9 s
Wall time: 10.9 s


In [445]:
fast_recs = embs_matrix

In [446]:
result['fast_lightfm_user_feats'] = result['user_id'].apply(lambda x: 
                                             v_to_item_id(np.argsort(fast_recs[userid_to_id[x]])[-5:]))

In [448]:
result.head()

,user_id,actual,fast_lightfm,lightfm,lightfm_user_feats,fast_lightfm_user_feats
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]"


In [449]:
# Все именно так. Полное совпадение с штатным predict-ом - и одинаковый прогноз для всех.

In [450]:
# Вобщем - NEED HELP

## Твик гиперпараметров

In [471]:
# Пришла в голову идея потвикать гиперпараметры
model = LightFM(no_components=30,
                loss='bpr',
                learning_rate=0.0001, 
                item_alpha=0.01, user_alpha=0.01, 
                random_state=100)

In [472]:
%%time
model.fit((csr_matrix(user_item_matrix).tocsr() > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, num_threads=4)

CPU times: user 1min 25s, sys: 205 ms, total: 1min 25s
Wall time: 1min 25s


In [473]:
precision_at_k(model, csr_matrix(user_item_matrix).tocsr(), 
                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                 k=5).mean()

0.37118852

In [476]:
%%time
result['lightfm_hparms'] = result['user_id'].apply(lambda x: 
                        v_to_item_id(
                            np.argsort(
                                model.predict(int(userid_to_id[x]), matrix_itemids, 
                                              user_features=csr_matrix(user_feat_lightfm.values).tocsr(), 
                                              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                              num_threads=4)
                            )[-6:])[::-1])

CPU times: user 3min 3s, sys: 1.13 s, total: 3min 4s
Wall time: 3min 6s


In [479]:
result.head()

,user_id,actual,fast_lightfm,lightfm,lightfm_user_feats,fast_lightfm_user_feats,lightfm_hparms
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 5668996, 6410462, ..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 1404121, 6410462, ..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 5850988, 6534178, ..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 1404121, 6410462, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 1404121, 5747420, ..."


In [480]:
# Так. Отбой - какое-то разнообразие получилось... Виной всему судя по всему был слишком большой learning_rate

In [481]:
# Ох. Пора уже упаковать это в src... что-то уже начинает запаривать копипейстить.
def pr_at_k(recommended, actual, k=5):
    flags = np.isin(recommended[:k], actual)
    return flags.sum() / len(recommended[:k])

v_precision_at_k = np.vectorize(pr_at_k)

In [482]:
v_precision_at_k(result.lightfm_hparms, result.actual).mean()

0.08256611165523996

Ха-ха ))
Извините, не смог удержаться...

In [484]:
# Кто-же интересно они - наши продукты - герои всех рекомендаций, ну кроме бузусловного лидера - 999999
item_features[item_features['item_id']==1082185]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
35576,1082185,2,PRODUCE,National,TROPICAL FRUIT,BANANAS,40 LB


In [485]:
item_features[item_features['item_id']==12518330]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
77201,12518330,2007,PRODUCE,National,TROPICAL FRUIT,BANANAS,8 OZ


In [486]:
item_features[item_features['item_id']==1404121]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
44796,1404121,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,


In [487]:
# Всем - бананов и бензина! Не самые плохие рекомендации я считаю )

### Взвесить матрицу

In [488]:
# Есть еще такие опции
weighted_matrix = bm25_weight(user_item_matrix)

In [489]:
model = LightFM(no_components=70,
                loss='bpr',
                learning_rate=0.0001, 
                item_alpha=0.01, user_alpha=0.01, 
                random_state=100)

In [490]:
%%time
model.fit((csr_matrix(user_item_matrix).tocsr() > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(weighted_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, num_threads=4)

CPU times: user 3min 12s, sys: 1.2 s, total: 3min 13s
Wall time: 3min 20s


In [491]:
precision_at_k(model, csr_matrix(user_item_matrix).tocsr(), 
                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                 k=5).mean()

0.25914368

In [495]:
%%time
result['lightfm_weighted'] = result['user_id'].apply(lambda x: 
                        v_to_item_id(
                            np.argsort(
                                model.predict(int(userid_to_id[x]), matrix_itemids, 
                                              user_features=csr_matrix(user_feat_lightfm.values).tocsr(), 
                                              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                              num_threads=4)
                            )[-5:])[::-1])

CPU times: user 2min 59s, sys: 650 ms, total: 3min
Wall time: 3min 1s


In [496]:
result.head()

,user_id,actual,fast_lightfm,lightfm,lightfm_user_feats,fast_lightfm_user_feats,lightfm_hparms,lightfm_weighted
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 5668996, 6410462, ...","[1126899, 1106523, 1070820, 1029743, 1065979]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 1404121, 6410462, ...","[1070820, 1029743, 1126899, 1106523, 1056212]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 5850988, 6534178, ...","[1126899, 1029743, 1106523, 1070820, 1056212]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 1404121, 6410462, ...","[1070820, 1029743, 1126899, 1106523, 1056212]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 1404121, 5747420, ...","[1126899, 1070820, 1106523, 1029743, 936830]"


In [497]:
v_precision_at_k(result.lightfm_weighted, result.actual).mean()

0.09147894221351617

In [498]:
# Медленно, но верно продвигаемся к отметке 10% )) взвешивание дает эффект. С ним в будущем можно повозиться

In [499]:
item_features[item_features['item_id']==1126899]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
40507,1126899,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA


In [500]:
item_features[item_features['item_id']==1106523]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
38262,1106523,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA


In [501]:
# На этот раз в лидерах - молоко. Вот теперь возникает серъезное желание пре-фильтровать таки популярные товары..

## Уберем еще и самые популярные товары

In [563]:
popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index()
popularity['user_id'] = popularity['user_id'] / data_train['user_id'].nunique()
popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()

In [564]:
data_train.loc[data_train['item_id'].isin(top_popular), 'item_id'] = 999999

/Users/postas/Anaconda/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [565]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

In [566]:
user_item_matrix.shape

(2499, 4992)

In [567]:
# Ну ок. Выкинулось еще 10 каких-то товаров

In [568]:
user_item_matrix = user_item_matrix.astype(float)

In [569]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [570]:
# Товарные фичи надо наверное пересобрать... Хотя - может даже и не надо.. 
# мы же убрали какие-то товары из интеракций - если на них есть фичи - это не должно наверное повлиять 
# поидее они просто не будут использоваться 

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
202291,69.0,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
397896,69.0,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,


In [571]:
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [572]:
# Взвесим
weighted_matrix = bm25_weight(user_item_matrix)

In [573]:
model = LightFM(no_components=70,
                loss='bpr',
                learning_rate=0.0001, 
                item_alpha=0.01, user_alpha=0.01, 
                random_state=100)

In [574]:
%%time
model.fit((csr_matrix(user_item_matrix).tocsr() > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(weighted_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, num_threads=4)

CPU times: user 2min 57s, sys: 1.02 s, total: 2min 58s
Wall time: 2min 59s


In [575]:
%%time
result['lightfm_wo_pop'] = result['user_id'].apply(lambda x: 
                        v_to_item_id(
                            np.argsort(
                                model.predict(int(userid_to_id[x]), matrix_itemids, 
                                              user_features=csr_matrix(user_feat_lightfm.values).tocsr(), 
                                              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                              num_threads=4)
                            )[-5:])[::-1])

CPU times: user 2min 55s, sys: 673 ms, total: 2min 55s
Wall time: 2min 56s


In [576]:
result.head()

,user_id,actual,fast_lightfm,lightfm,lightfm_user_feats,fast_lightfm_user_feats,lightfm_hparms,lightfm_weighted,lightfm_wo_pop
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 5668996, 6410462, ...","[1126899, 1106523, 1070820, 1029743, 1065979]","[1126899, 1106523, 1070820, 1065979, 936830]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 1404121, 6410462, ...","[1070820, 1029743, 1126899, 1106523, 1056212]","[1106523, 1070820, 1126899, 936830, 1056212]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 5850988, 6534178, ...","[1126899, 1029743, 1106523, 1070820, 1056212]","[1070820, 1126899, 1106523, 1056212, 936830]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 1404121, 6410462, ...","[1070820, 1029743, 1126899, 1106523, 1056212]","[1106523, 1126899, 1070820, 936830, 1065979]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[5703832, 397896, 1404121, 1082185, 999999]","[5703832, 397896, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[397896, 2690723, 1404121, 1082185, 999999]","[1082185, 999999, 12518330, 1404121, 5747420, ...","[1126899, 1070820, 1106523, 1029743, 936830]","[1126899, 1070820, 1106523, 1065979, 936830]"


In [577]:
v_precision_at_k(result.lightfm_wo_pop, result.actual).mean()

0.05171400587659158

In [ ]:
# Не вариант, похоже.